In [ ]:
# SELECT timestamp, text, retweets FROM tweets
# ORDER BY tweets.retweets
# LIMIT 1000000
# INTO OUTFILE
# 'ruta/nombre.csv'
# FIELDS TERMINATED BY ';'
# OPTIONALLY ENCLOSED BY '\"'
# LINES TERMINATED BY '\n';


## en windows LINES TERMINATED BY '\r';
## Código SQL para reducir la dimensión de 17 millones a solo el primer millon 
##con más retweets


In [ ]:
import pandas_datareader.data as web
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.graph_objects as go
import plotly.express as px
import plotly.express as px
from plotly.subplots import make_subplots

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
columns=['timestamp','text']
datos=pd.read_excel('drive/MyDrive/Colab Notebooks/p2.xlsx', sheet_name="Hoja2", usecols=columns)
datos.info()

In [ ]:
#fecha más antigua
datos.timestamp.min()

In [ ]:
#fecha más reciente
datos.timestamp.max()

In [ ]:
#eliminar na's
datos.dropna(inplace=True)

In [ ]:
# Libreria de análisis de sentimientos
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

In [ ]:
#Generar variable con el compound score
datos["Score"]=datos.text.apply(lambda x: sid.polarity_scores(x)['compound'])

In [ ]:
datos.head(5)

In [ ]:
datos.tail(5)

In [ ]:
#Resumen Base de datos
datos.describe()

In [ ]:
#Volver variable timestamp el index de la base de datos
datos.index=datos.timestamp

In [ ]:
# Agregar los datos y encontrar promedio del compound score por dia
scores=datos['Score'].resample("d").mean()
scores.fillna(0, inplace=True)
datos.describe()

In [ ]:
fig = plt.figure(1, figsize=(9, 6))
plt.plot(scores)

In [ ]:
#installing yfinance (Colab)
!pip install yfinance
#pip install yfinance --upgrade --no-cache-dir
import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()

In [ ]:
#Univariate time series
#df_btc= web.DataReader("BTC-USD", 'yahoo')#Use if yfinance does not work
print("-------------------------------- DATA -------------------------------")
df_btc = pdr.get_data_yahoo(["BTC-USD"])
df_btc.head(5)

In [ ]:
# generar un merge entre los datos de bitcoin y la serie del compound score
p2=pd.concat([df_btc,scores], axis=1, join="inner")
p2.head(5)

In [ ]:
#Variable del cambio en el precio del bitcoin con respecto al día anterior
p2['Cambio']=p2.Close-p2.Open

In [ ]:
# Definir función para crear variable de interes "1" si el precio subio ese día
def subida(x):
  if x>0:
    return 1
  else:
    return 0

In [ ]:
p2['dummy']=p2.Cambio.apply(subida)
p2.head()

In [ ]:
#generar variables X Y para el modelo
X=np.array(p2.Score).reshape(-1,1)
y=np.array(p2.dummy)
X.shape

In [ ]:
#librerias para el modelo logistico
from sklearn import linear_model
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [ ]:
#Generación de semilla para usar validación cruzada
validation_size=0.20
seed=7
X_train, X_val, y_train, y_val=model_selection.train_test_split(X, y, test_size=validation_size, random_state=seed)

In [ ]:
#Modelo con validación cruzada
model = linear_model.LogisticRegression()
model.fit(X,y)
name='Logistic Regression'
kfold = model_selection.KFold(n_splits=10, random_state=seed)
cv_results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy')
msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
print(msg)

In [ ]:
#Accuracy
predictions = model.predict(X_val)
print(accuracy_score(y_val, predictions))

In [ ]:
#Modelo con datos de entrenamiento y predicciones con validación
model.fit(X_train,y_train)
predictions = model.predict(X_val)
print(accuracy_score(y_val, predictions))

In [ ]:
#Resumen del modelo
print(classification_report(y_val, predictions))